In [47]:
from datasets import load_dataset
test = load_dataset("/home/adil/Documents/TuE/q4/KnowledgeEngineering/knowledgeEngineering/new-recipe-code/dataset", split="train")
test

Found cached dataset csv (/home/adil/.cache/huggingface/datasets/csv/dataset-4afc3749bed64de0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER'],
    num_rows: 2231142
})

In [214]:
import pandas as pd
df: pd.DataFrame = pd.DataFrame(test[:1000])

In [215]:
df['ingredients'][2]

'["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"]'

In [239]:
import json
import re
from fractions import Fraction


import typing
unitConversions: dict = {
    'c.': '236.588 ml',
    'tsp.': '4.92892 ml',
    'tbsp.': '14.7868 ml',
    'oz.': '29.5735 ml',
    'pt.': '473.176 ml',
    'qt.':' 946.353 ml',
    'lb.': '453.592 grams',
    'gal.': '3785.41 ml',
    ' oz': ' 29.5735 ml',
}

def replace_units(text: str) -> str:
    # replaces american units with european units
    text= text.lower()
    for key, value in unitConversions.items():
        text = text.replace(key, value)
    return text
def replace_fractions(text:str)-> str:
    # replaces fractions with decimals
    text = text.lower()
    text = re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, text)
    return text
def frac2string(s):
    # helper function for replace_fractions
    i, f = s.groups(0)
    f = Fraction(f)
    return str(int(i) + float(f))


def convert_and_multiply_units(text: str) -> list:
    # converts units to floats within strings and multiplies them
    # e.g 1 0.5 0.24 ml becomes 0.12 ml
    ingredients: list = json.loads(text)
    ingredient_list = []
    for ingredient in ingredients: # ingredient is a string with units and one specific ingredient
        ingredient_split: list[str] = re.split(r'\s+|\)|\(', ingredient)
        filtered_result = [item for item in ingredient_split if item]
        converted_values: list= []
        for index, item in enumerate(filtered_result):
            try: # try to turn it into a float and multiply
                converted_value = float(item)
                if len(converted_values) >= 1 and isinstance(converted_values[-1], float):
                    converted_value = converted_values[-1] * converted_value # multiply latest float with current
                    converted_values.pop(-1) # remove latest value, since were using it to multiply
                    converted_values.append(float(converted_value)) 
                else:
                    converted_values.append(converted_value)   
            except ValueError:
                converted_values.append(item)
                pass
            except  IndexError:
                pass
        ingredient_string = " ".join(str(item) for item in converted_values) # join the list back together    
        ingredient_list.append(ingredient_string)
    return ingredient_list


df['ingredients'] = df['ingredients'].apply(lambda text: replace_units(text))
df['ingredients'] = df['ingredients'].apply(lambda text: replace_fractions(text))
df['ingredients'] = df['ingredients'].apply(lambda text: convert_and_multiply_units(text))


In [240]:
df['ingredients']

0      [236.588 ml firmly packed brown sugar, 118.294...
1      [1.0 small jar chipped beef, cut up, 4.0 boned...
2      [946.352 ml pkg. frozen corn, 236.588 ml pkg. ...
3      [1.0 large whole chicken, 621.0435 ml cans chi...
4      [236.588 ml peanut butter, 177.441 ml graham c...
                             ...                        
995    [3.0 heath bars, chopped fine, 1.0 medium cont...
996    [236.588 ml brown sugar, 78.86266666666666 ml ...
997    [453.592 grams ground beef, 236.588 ml soft br...
998    [907.184 grams lamb, cut in 1-inch cubes, 473....
999    [2-3 29.5735 ml boxes peach jello, 1.0 large j...
Name: ingredients, Length: 1000, dtype: object

In [ ]:
def multiply_units(text: str):
    ingredients: list = json.loads(text)
    for ingredient in ingredients:
        ingredient_split: list[str] = re.split(r'\s+|\)|\(', ingredient)
        filtered_result = [item for item in ingredient_split if item]
        print(filtered_result)
        for index, item in enumerate(filtered_result):
            try:
                filtered_result[index] = float(item)
                print(filtered_result)
                    
            except ValueError as e:
                pass

In [204]:
testlist = [1,2,3,4,5,6,7,8,9,10]

10


In [ ]:
import networkx as nx
from pyvis.network import Network
from IPython.core.display import HTML

# Enable the pyvis interactive display in the Jupyter notebook
from pyvis import network as net
net.template = "notebook"
net.NOTEBOOK = True


# Create a NetworkX graph
G = nx.Graph()
node_list = []
edge_list = []
for key, value in dictTopic.items():
    if data[int(key)].find('country') != None:
        country = data[int(key)].find('country').text
        country = country.replace(f'</country>', "")

        topic = dictTopic[key].replace(f'<topic type="str">', "")
        topic = topic.replace(f'</topic>', "")
        node_list.append(country)
        node_list.append(topic)
        edge_list.append((country, topic))
    else:
        #print("we didn't find a country for this specific article: ", data[int(key)].attrib['id'])
        pass
    
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

# Create a pyvis network object
net = Network(height="500px", width="100%", bgcolor="#222222", font_color="white", notebook=True)

# Add nodes and edges to the pyvis network object, and set tooltips for each node
for node in G.nodes():
    tooltip = "{}".format(node)
    net.add_node(node, title=tooltip)
for edge in G.edges():
    net.add_edge(*edge, title="hasTopic")
print("Please zoom-in using your scrollwheel and hover over the nodes and edges")
# Display the interactive network graph with tooltips in the Jupyter notebook

net.show("example.html")